In [1]:
%env APP_ENV=prod

env: APP_ENV=prod


In [7]:
import time

import polars as pl
from sqlalchemy import select

from src.db.database import get_db_session
from src.db.models import FundSnapshotDao


def time_cost(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Time cost: {end_time - start_time} seconds")
        return result

    return wrapper


@time_cost
def test_read():
    with get_db_session() as session:
        stmt = select(FundSnapshotDao)
        result = session.execute(stmt).scalars().all()
        print(result[0])
        df = pl.DataFrame(result)
    return df


In [8]:
df = test_read()

FundSnapshotDao(id=1, fund_id=153, fund_asset=Decimal('4994044.287400'), is_id=124, is_name='高频择时', is_type='高频择时', bsp_id=None, bsp_name=None, bsp_type='stock', sa_type='mercury9', is_wt=0.42133333333333345, bsp_wt=1.0, basis_pos_pct=None, mv_wt=0.4213333333333335, bsp_asset=Decimal('2104157.326425'), bsp_mv=2104157.326424534, bsp_share=None, future_price=None, target_is_expo=0.42133333333333345, trading_day=datetime.date(2025, 5, 29), updater='AUTO-20250529084934701981')
Time cost: 0.7290666103363037 seconds


In [14]:
import datetime

In [18]:
df.filter(pl.col("trading_day") == datetime.date(2025, 7, 18)).filter(pl.col("fund_id") == 1)

id,fund_id,fund_asset,is_id,is_name,is_type,bsp_id,bsp_name,bsp_type,sa_type,is_wt,bsp_wt,basis_pos_pct,mv_wt,bsp_asset,bsp_mv,bsp_share,future_price,target_is_expo,trading_day,updater
i64,i64,"decimal[*,6]",i64,str,str,i64,str,str,str,f64,f64,f64,f64,"decimal[*,6]",f64,i64,f64,f64,date,str
14042,1,38183390.847800,64,"""500指增""","""500指增""",1512,"""IC_long_comm""","""future""","""alpha""",0.997073,0.031299,1.0,0.031207,1191600.000000,1.1916e6,1,1.1916e6,0.997073,2025-07-18,"""AUTO-20250718084915CcTx6b"""
14043,1,38183390.847800,64,"""500指增""","""500指增""",100,"""500_comm""","""stock""","""alpha""",0.997073,0.968701,1.0,0.965865,36880012.654000,3.6880e7,null,null,0.997073,2025-07-18,"""AUTO-20250718084915CcTx6b"""
14044,1,38183390.847800,1,"""现金""","""现金""",null,null,"""stock""","""_cash""",0.002927,1.0,null,0.0,111778.193800,0.0,null,null,0.002927,2025-07-18,"""AUTO-20250718084915CcTx6b"""


In [16]:
df.filter(pl.col("trading_day") == datetime.date(2025, 7, 18)).group_by("fund_id").agg(
    pl.struct(pl.col("is_id"), pl.col("is_name"), pl.col("is_type")).alias("is_info")
).sort("fund_id")

fund_id,is_info
i64,list[struct[3]]
1,"[{64,""500指增"",""500指增""}, {64,""500指增"",""500指增""}, {1,""现金"",""现金""}]"
18,"[{3,""1000对冲"",""中性""}, {3,""1000对冲"",""中性""}, … {31,""科创板指增"",""科创板指增""}]"
48,"[{97,""股票精选"",""股票精选""}, {97,""股票精选"",""股票精选""}, {1,""现金"",""现金""}]"
49,"[{64,""500指增"",""500指增""}, {64,""500指增"",""500指增""}, {1,""现金"",""现金""}]"
51,"[{124,""高频择时"",""高频择时""}, {123,""高频择时"",""高频择时""}, … {142,""高频择时测试"",""高频择时测试""}]"
…,…
445,"[{96,""股票精选"",""股票精选""}, {90,""科技创新精选"",""科技创新精选""}]"
446,"[{99,""300对冲"",""中性""}, {99,""300对冲"",""中性""}, {90,""科技创新精选"",""科技创新精选""}]"
448,"[{1,""现金"",""现金""}, {90,""科技创新精选"",""科技创新精选""}]"


In [19]:
df.filter(pl.col("trading_day") == datetime.date(2025, 7, 18)).pivot(
    on="fund_id", index="is_id", values="is_wt"
)

ComputeError: found multiple elements in the same group, please specify an aggregation function